In [1]:
!pip install --upgrade pip
!pip install 'farm-haystack'
!pip install datasets==1.0.2
!pip install tqdm==4.57.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.1/713.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.9/299.9 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0

In [2]:
import pandas as pd
pd.set_option("display.max_colwidth", -1)

<ipython-input-2-b1bfa594df05>:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


In [3]:
# connect your personal google drive to store dataset and trained model
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


**Preprocessing the Biology textbook Chapter 2 - Chemistry of life text**

In [5]:
#--Using Haystack API to 
from haystack import Pipeline
from haystack.nodes import PreProcessor,TextConverter

processor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
    split_overlap=5
)

converter = TextConverter();
doc = converter.convert(file_path='/content/gdrive/My Drive/CS 677 Project/dataset/chemistryoflife_v3.txt', meta=None)
docs = processor.process(doc)

Preprocessing:   0%|          | 0/1 [00:00<?, ?docs/s]

In [6]:
df = pd.DataFrame(docs)
df.shape

(22, 7)

In [7]:
df.head()
df.to_excel("/content/gdrive/My Drive/CS 677 Project/dataset/chemistryoflife_all_v3.xlsx",
             sheet_name='Sheet_name_1') 

In [8]:
chem = df['content']

In [9]:
chem.shape

(22,)

In [10]:
chem.head()

0    Atoms are the building blocks that come together through chemical bonding to...
1    The elements carbon, hydrogen, nitrogen, oxygen, sulfur, and phosphorus are ...
2    At its most fundamental level, life is made up of matter. Matter is any subs...
3    Each element is typically designated by a single capital letter, or two lett...
4    The four elements common to all living organisms are oxygen (O), carbon (C),...
Name: content, dtype: object

In [11]:
chem.to_excel("/content/gdrive/My Drive/CS 677 Project/dataset/chemistryoflife_v3.xlsx",
             sheet_name='Sheet_name_1')  

**Preparing the SQUAD dataset for validation and comparison**

In [ ]:
from datasets import load_dataset
train_dataset = load_dataset('squad', split='train')
valid_dataset = load_dataset('squad', split='validation')

Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/1244d044b266a5e4dbd4174d23cb995eead372fbca31a03edc3f8a132787af41)
Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/1244d044b266a5e4dbd4174d23cb995eead372fbca31a03edc3f8a132787af41)


In [ ]:
from pprint import pprint 
sample_validation_dataset = next(iter(valid_dataset))
pprint (sample_validation_dataset)

{'answers': {'answer_start': [177, 177, 177],
             'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']},
 'context': 'Super Bowl 50 was an American football game to determine the '
            'champion of the National Football League (NFL) for the 2015 '
            'season. The American Football Conference (AFC) champion Denver '
            'Broncos defeated the National Football Conference (NFC) champion '
            'Carolina Panthers 24–10 to earn their third Super Bowl title. The '
            "game was played on February 7, 2016, at Levi's Stadium in the San "
            'Francisco Bay Area at Santa Clara, California. As this was the '
            '50th Super Bowl, the league emphasized the "golden anniversary" '
            'with various gold-themed initiatives, as well as temporarily '
            'suspending the tradition of naming each Super Bowl game with '
            'Roman numerals (under which the game would have been known as '
            '"Super

In [ ]:
context = sample_validation_dataset['context']
question = sample_validation_dataset['question']
answer = sample_validation_dataset['answers']['text'][0]

print ("context: ",context)
print ("question: ",question)
print ("answer: ",answer)

context:  Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
question:  Which NFL team represented the AFC at Super Bowl 50?
answer:  Denver Broncos


In [ ]:
df_train = pd.DataFrame( columns = ['context', 'answer','question'])
df_validation = pd.DataFrame( columns = ['context', 'answer','question'])
print (df_validation)
print (df_train)

Empty DataFrame
Columns: [context, answer, question]
Index: []
Empty DataFrame
Columns: [context, answer, question]
Index: []


In [ ]:
from tqdm.notebook import tqdm
count_long = 0
count_short = 0

        
for index,val in enumerate(tqdm(train_dataset)):
    passage = val['context']
    question = val['question']
    answer = val['answers']['text'][0]
    no_of_words = len(answer.split())
    if no_of_words >= 7:
        count_long = count_long + 1
        continue
    else:
        df_train.loc[count_short]= [passage] + [answer] + [question] 
        count_short = count_short + 1       

print ("count_long train dataset: ",count_long)
print ("count_short train dataset: ",count_short)

  0%|          | 0/87599 [00:00<?, ?it/s]

count_long train dataset:  8935
count_short train dataset:  78664


In [ ]:
count_long = 0
count_short = 0

        
for index,val in enumerate(tqdm(valid_dataset)):
    passage = val['context']
    question = val['question']
    answer = val['answers']['text'][0]
    no_of_words = len(answer.split())
    if no_of_words >= 7:
        count_long = count_long + 1
        continue
    else:
        df_validation.loc[count_short]= [passage] + [answer] + [question] 
        count_short = count_short + 1       

print ("count_long validation dataset: ",count_long)
print ("count_short validation dataset: ",count_short)

  0%|          | 0/10570 [00:00<?, ?it/s]

count_long validation dataset:  918
count_short validation dataset:  9652


In [ ]:
from sklearn.utils import shuffle
df_train = shuffle(df_train)
df_validation = shuffle(df_validation)

In [ ]:
print (df_train.shape)
print (df_validation.shape)

(78664, 3)
(9652, 3)


In [ ]:
df_train.head()

,context,answer,question
10866,"In 1919 Nancy Astor was elected the first ever female member of parliament to take office in the British Houses of Parliament for the constituency of Plymouth Sutton. Taking over office from her husband Waldorf Astor, Lady Astor was a vibrantly active campaigner for her resident constituents . Plymouth was granted city status on 18 October 1928. The city's first Lord Mayor was appointed in 1935 and its boundaries further expanded in 1967 to include the town of Plympton and the parish of Plymstock.",Plymstock,What parish was incorporated into Plymouth in 1967?
70217,"In the early hours of 7 December (Hawaiian time), Japan launched a major surprise carrier-based air strike on Pearl Harbor without explicit warning, which crippled the U.S. Pacific Fleet, leaving eight American battleships out of action, 188 American aircraft destroyed, and 2,403 American citizens dead. At the time of the attack, the U.S. was not officially at war anywhere in the world, which means that the people killed or property destroyed at Pearl Harbor by the Japanese attack had a non-combatant status.[nb 11] The Japanese had gambled that the United States, when faced with such a sudden and massive blow, would agree to a negotiated settlement and allow Japan free rein in Asia. This gamble did not pay off. American losses were less serious than initially thought: The American aircraft carriers, which would prove to be more important than battleships, were at sea, and vital naval infrastructure (fuel oil tanks, shipyard facilities, and a power station), submarine base, and signals intelligence units were unscathed. Japan's fallback strategy, relying on a war of attrition to make the U.S. come to terms, was beyond the IJN's capabilities.",188,How many aircraft did the United States lose in the attack on Pearl Harbor by Japan?
53596,"Like other common colors, green has several completely opposite associations. While it is the color most associated by Europeans and Americans with good health, it is also the color most often associated with toxicity and poison. There was a solid foundation for this association; in the nineteenth century several popular paints and pigments, notably verdigris, vert de Schweinfurt and vert de Paris, were highly toxic, containing copper or arsenic.[d] The intoxicating drink absinthe was known as ""the green fairy"".",copper or arsenic,Why were popular paints and pigments in the nineteenth century highly toxic?
13393,"Although the importation of slaves to St Helena had been banned in 1792, the phased emancipation of over 800 resident slaves did not take place until 1827, which was still some six years before the British Parliament passed legislation to ban slavery in the colonies.",British Parliament,Who banned slavery in their colonies?
28204,"With the arrival of Francisco Vázquez de Coronado, a Spanish conquistador, the first recorded history of encounter between Europeans and Native Americans in the Great Plains occurred in Texas, Kansas and Nebraska from 1540-1542. In that same time period, Hernando de Soto crossed a west-northwest direction in what is now Oklahoma and Texas. Today this is known as the De Soto Trail. The Spanish thought the Great Plains were the location of the mythological Quivira and Cíbola, a place said to be rich in gold.",Hernando de Soto,who crossed a section of oklahoma and texas?


In [ ]:
train_save_path = '/content/gdrive/My Drive/CS 677 Project/dataset/squad_t5_train.csv'
validation_save_path = '/content/gdrive/My Drive/CS 677 Project/dataset/squad_t5_val.csv'
df_train.to_csv(train_save_path, index = False)
df_validation.to_csv(validation_save_path, index = False)